In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import random
from torch.utils.data import DataLoader, Dataset, Subset, random_split
from torchvision import datasets, transforms
from torchvision.datasets import SBDataset
from tqdm import tqdm

C:\Users\thoma\miniconda3\envs\torch_gpu\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [8]:
class SBDSegmentationDataset(Dataset):
    def __init__(self, base_dataset, img_transform=None, mask_transform=None):
        self.base = base_dataset
        self.img_transform = img_transform
        self.mask_transform = mask_transform

    def __len__(self):
        return len(self.base)

    def __getitem__(self, idx):
        img, mask = self.base[idx]

        if self.img_transform:
            img = self.img_transform(img)
        if self.mask_transform:
            mask = self.mask_transform(mask)

        return img, mask

In [13]:
img_transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor()
])

mask_transform = transforms.Compose([
    transforms.Resize((256,256),
    interpolation=transforms.InterpolationMode.NEAREST),
    transforms.PILToTensor()
])

train_base = SBDataset(
    root='./data',
    image_set='train',
    mode='segmentation',
    download=False
)

val_base = SBDataset(
    root='./data',
    image_set='val',
    mode='segmentation',
    download=False
)

train_dataset = SBDSegmentationDataset(train_base, img_transform, mask_transform)
val_dataset = SBDSegmentationDataset(val_base, img_transform, mask_transform)

train_subset = Subset(train_dataset, range(4000))

dataset_size = len(train_subset)
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size

train_ds, val_ds = random_split(train_subset, [train_size, val_size], generator=torch.Generator().manual_seed(42))

In [14]:
train_loader = DataLoader(
    train_ds,
    batch_size=8,
    shuffle=True,
    pin_memory=True,
    persistent_workers=False
)

val_loader = DataLoader(
    val_ds,
    batch_size=8,
    shuffle=False,
    pin_memory=True,
    persistent_workers=False
)